In [1]:
import pandas as pd
import random
import numpy

from bs4 import BeautifulSoup
import requests
import re
import json
import csv

from selenium import webdriver  
from webdriver_manager.chrome import ChromeDriverManager  
from selenium.webdriver.common.keys import Keys  
from selenium.webdriver.support.ui import Select 

from time import sleep

pd.set_option('display.max_columns', None)

C:\Users\Isabel\AppData\Local\Temp\ipykernel_14868\3864169933.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [7]:
import json
with open('JsonDrama.json', 'r') as fichero:
    busqueda = json.load(fichero)

In [8]:
busqueda

[['tt0102362', 'drama', 'Istota', 'movie', 'desconocido', 'desconocido'],
 ['tt0113092', 'drama', 'For the Cause', 'movie', 2002, 5],
 ['tt0113086', 'drama', 'Florentino y el diablo', 'movie', 2000, 4],
 ['tt0116748', 'drama', 'Karobaar: The Business of Love', 'movie', 2000, 9],
 ['tt0115874',
  'drama',
  'Chateaubriand - Cabeça de Paraíba',
  'short',
  2000,
  12],
 ['tt0116391', 'drama', 'Gang', 'movie', 2000, 4],
 ['tt0115937', 'drama', 'Consequence', 'movie', 'desconocido', 'desconocido'],
 ['tt0118694', 'drama', 'Fa yeung nin wah', 'movie', 2001, 3],
 ['tt0118710', 'drama', 'Beyond the Pale', 'movie', 2000, 9],
 ['tt0118852', 'drama', 'Chinese Coffee', 'movie', 2000, 9],
 ['tt0119866', 'drama', 'Passionnément', 'movie', 2000, 5],
 ['tt0119495', 'drama', 'Love 101', 'movie', 2000, 2],
 ['tt0119966', 'drama', 'Pushing the Envelope', 'movie', 2001, 4],
 ['tt0120485',
  'drama',
  'Way Past Cool',
  'movie',
  'desconocido',
  'desconocido'],
 ['tt0120202', 'drama', 'State and Main'

In [2]:
valoraciones = {}

In [42]:
errores = []

In [43]:

driver = webdriver.Chrome()
driver.get('https://www.google.com/')
driver.maximize_window()
sleep(2)

driver.find_element('css selector', '#W0wltc > div').click()
sleep(2)

for pelicula in busqueda:
    valoracion = [f'{pelicula[0]}',]
    try:
        driver.find_element('css selector', '#APjFqb').send_keys(pelicula[0],' imdb', Keys.ENTER)
        
        nota_imdb = driver.find_element('css selector', '#rso > div:nth-child(1) > div > div > div:nth-child(3) > div > span:nth-child(2)').text
        valoracion.append(nota_imdb)

        driver.find_element('css selector', '#APjFqb').clear()

        driver.find_element('css selector', '#APjFqb').send_keys(pelicula[0],' imdb', Keys.ENTER)

        
    except:
        errores.append(pelicula)

    driver.find_element('css selector', '#APjFqb').clear()


    

nuevo codigo - info imdb

In [17]:
busqueda2 = ['tt0119806', 'tt0123003', 'tt0273870', 'tt0132245']

In [21]:
with open('../Fase1_datos/lista_drama.json', 'r') as fichero:
    busqueda_drama = json.load(fichero)

In [24]:
busqueda = busqueda_drama[:10]

In [34]:
busqueda

[['tt0113092', 'drama', 'For the Cause', 'movie', 2002, 5],
 ['tt0113086', 'drama', 'Florentino y el diablo', 'movie', 2000, 4],
 ['tt0116748', 'drama', 'Karobaar: The Business of Love', 'movie', 2000, 9],
 ['tt0115874',
  'drama',
  'Chateaubriand - Cabeça de Paraíba',
  'short',
  2000,
  12],
 ['tt0116391', 'drama', 'Gang', 'movie', 2000, 4],
 ['tt0118694', 'drama', 'Fa yeung nin wah', 'movie', 2001, 3],
 ['tt0118710', 'drama', 'Beyond the Pale', 'movie', 2000, 9],
 ['tt0118852', 'drama', 'Chinese Coffee', 'movie', 2000, 9],
 ['tt0119866', 'drama', 'Passionnément', 'movie', 2000, 5],
 ['tt0119495', 'drama', 'Love 101', 'movie', 2000, 2]]

In [2]:
# llegar a web:
driver = webdriver.Chrome()
driver.get('https://www.imdb.com/')
driver.maximize_window()
sleep(3)


# rechazar cookies
driver.find_element('css selector', '#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-hLBbgP.sc-ftTHYK.dcvrLS.dufgkr.ecppKW').click()
sleep(2)


# variables donde iremos almacenando todo
info_por_peli = []
fallos = []
lista_tuplas = []


# entrando en cada título
for peli in busqueda:
    lista_info = [f'{peli[0]}',]

    try:
    # metemos el ID en el buscador
        driver.find_element('css selector', '#suggestion-search').send_keys(peli[0], Keys.ENTER)
        sleep(3)

        # extraemos info
        info = driver.find_element('xpath' , f'//*[@id="__next"]/main/div/section[1]/section/div[3]/section').text
        lista_info.append(info.split('\n'))

        info_por_peli.append(lista_info)
    except:
        fallos.append(peli)


# recogida de los datos
for peli in info_por_peli:
    tupla_datos = [i[0],]
    
    for indice, linea in enumerate(peli[1]):
        patron = re.compile(r'([a-z])([A-Z])')

        # directores    
        if 'Dirección' in linea:
                                
            directores = patron.sub(r'\1, \2', peli[1][indice+1])
            tupla_datos.append(directores)            


        # guionistas
        elif 'Guión' in linea:
            
            guionistas = patron.sub(r'\1, \2', peli[1][indice+1])
            tupla_datos.append(guionistas)
                
        
        # nota y duracion
        elif 'PUNTUACIÓN EN IMDb' in linea:
            tupla_datos.append(float(peli[1][indice+1].replace(',', '.')))
            tupla_datos.append(peli[1][indice-1])
            
                    
        # argumento
        elif 'Añade un argumento en tu idioma' in linea:
            tupla_datos.append(peli[1][indice+1])
    
    lista_tuplas.append(tuple(tupla_datos))
        

        




KeyboardInterrupt: 

In [50]:
info_por_peli

[["['tt0113092', 'drama', 'For the Cause', 'movie', 2002, 5]",
  ['Reparto y equipo',
   'Reseñas de usuarios',
   'Curiosidades',
   'IMDbPro',
   'Todos los temas',
   'Por la causa',
   'Título original: For the Cause',
   '2000',
   'R',
   '1h 40min',
   'PUNTUACIÓN EN IMDb',
   '3,4',
   '/10',
   '841',
   'TU PUNTUACIÓN',
   'Puntuar',
   'Acción',
   'Aventura',
   'Drama',
   'Añade un argumento en tu idioma',
   'Earth is in a state of constant war and two colonies of humans have been sent to a far away world. The colonists create a new civilization but have inherited the worst traits of their forebears and war breaks out leaving the new cities in ruins. It falls to a bunch of teens to bring peace to the colony - but will they?',
   'Dirección',
   'David DouglasTim Douglas',
   'Guión',
   'David DouglasChristopher Salazar',
   'Reparto principal',
   'Dean CainThomas Ian GriffithJustin Whalin',
   'Ver la información de la producción en IMDbPro',
   'Añade a mi lista de se

In [57]:
pd.Series(["Dirección"]).isin(info_por_peli[0][1])

0    True
dtype: bool

In [61]:
a = pd.Series(info_por_peli[0][1])
a[a=='Dirección'].index[0]

21

In [76]:
info_por_peli[0]

["['tt0113092', 'drama', 'For the Cause', 'movie', 2002, 5]",
 ['Reparto y equipo',
  'Reseñas de usuarios',
  'Curiosidades',
  'IMDbPro',
  'Todos los temas',
  'Por la causa',
  'Título original: For the Cause',
  '2000',
  'R',
  '1h 40min',
  'PUNTUACIÓN EN IMDb',
  '3,4',
  '/10',
  '841',
  'TU PUNTUACIÓN',
  'Puntuar',
  'Acción',
  'Aventura',
  'Drama',
  'Añade un argumento en tu idioma',
  'Earth is in a state of constant war and two colonies of humans have been sent to a far away world. The colonists create a new civilization but have inherited the worst traits of their forebears and war breaks out leaving the new cities in ruins. It falls to a bunch of teens to bring peace to the colony - but will they?',
  'Dirección',
  'David DouglasTim Douglas',
  'Guión',
  'David DouglasChristopher Salazar',
  'Reparto principal',
  'Dean CainThomas Ian GriffithJustin Whalin',
  'Ver la información de la producción en IMDbPro',
  'Añade a mi lista de seguimiento',
  '9',
  'Reseñas 

In [88]:
for pelicula in info_por_peli:
    tupla_datos = [pelicula[0]
    pelicula_Ser = pd.Series(pelicula[1])

    for elemento in ['Dirección', 'Guión', 'PUNTUACIÓN EN IMDb', 'Añade un argumento en tu idioma']:

        if pd.Series(elemento).isin(pelicula_Ser)[0]:
            indice = pelicula_Ser[pelicula_Ser == elemento].index[0]

            if elemento in ['Dirección', 'Guión']:
                nombre = patron.sub(r'\1, \2', pelicula[1][indice+1])
                tupla_datos.append(nombre)

            else:
                tupla_datos.append('desconocido')
            
            if elemento == 'PUNTUACIÓN EN IMDb':
                tupla_datos.append(float(pelicula[1][indice+1].replace(',', '.')))
            else:
                tupla_datos.append('desconocido')

            if elemento == 'argumento':
                tupla_datos.append(pelicula[1][indice+1])
            else:
                tupla_datos.append('desconocido')
                



In [89]:
tupla_datos

['tt0119495',
 'Adrian Fulle',
 'Adrian Fulle, David J. Miller, Daniel Bingley',
 5.0,
 'A group of raucous college buds living the Dorm life, decide to have a blowout while most of their friends are away for the Thanksgiving holiday. With LOVE 101 being the toughest course this semester, will they pass the final exam?']

In [ ]:


# recogida de los datos
for i in info_por_peli:
    tupla_datos = [i[0],]
    
    for indice, linea in enumerate(i[1]):
        patron = re.compile(r'([a-z])([A-Z])')

        # directores    
        if 'Dirección' in linea:
                                
            directores = patron.sub(r'\1, \2', i[1][indice+1])
            tupla_datos.append(directores)         


        # guionistas
        elif 'Guión' in linea:
            
            guionistas = patron.sub(r'\1, \2', i[1][indice+1])
            tupla_datos.append(guionistas)
                
        
        # nota y duracion
        elif 'PUNTUACIÓN EN IMDb' in linea:
            tupla_datos.append(float(i[1][indice+1].replace(',', '.')))
            tupla_datos.append(i[1][indice-1])
            
                    
        # argumento
        elif 'Añade un argumento en tu idioma' in linea:
            tupla_datos.append(i[1][indice+1])
    
    lista_tuplas.append(tuple(tupla_datos))

In [46]:
len(info_por_peli)

10

In [49]:
info_por_peli[0]

["['tt0113092', 'drama', 'For the Cause', 'movie', 2002, 5]",
 ['Reparto y equipo',
  'Reseñas de usuarios',
  'Curiosidades',
  'IMDbPro',
  'Todos los temas',
  'Por la causa',
  'Título original: For the Cause',
  '2000',
  'R',
  '1h 40min',
  'PUNTUACIÓN EN IMDb',
  '3,4',
  '/10',
  '841',
  'TU PUNTUACIÓN',
  'Puntuar',
  'Acción',
  'Aventura',
  'Drama',
  'Añade un argumento en tu idioma',
  'Earth is in a state of constant war and two colonies of humans have been sent to a far away world. The colonists create a new civilization but have inherited the worst traits of their forebears and war breaks out leaving the new cities in ruins. It falls to a bunch of teens to bring peace to the colony - but will they?',
  'Dirección',
  'David DouglasTim Douglas',
  'Guión',
  'David DouglasChristopher Salazar',
  'Reparto principal',
  'Dean CainThomas Ian GriffithJustin Whalin',
  'Ver la información de la producción en IMDbPro',
  'Añade a mi lista de seguimiento',
  '9',
  'Reseñas 

In [48]:
lista_tuplas

[("['tt0113092', 'drama', 'For the Cause', 'movie', 2002, 5]",
  'desconocido',
  'desconocido',
  'desconocido',
  'desconocido',
  'desconocido',
  'desconocido',
  'desconocido',
  'desconocido',
  'desconocido',
  'desconocido',
  'desconocido',
  'desconocido',
  'desconocido',
  'desconocido',
  'desconocido',
  'desconocido',
  'desconocido',
  'desconocido',
  'desconocido',
  'desconocido',
  'desconocido',
  'desconocido',
  3.4,
  '1h 40min',
  'desconocido',
  'desconocido',
  'desconocido',
  'desconocido',
  'desconocido',
  'desconocido',
  'desconocido',
  'desconocido',
  'desconocido',
  'desconocido',
  'desconocido',
  'desconocido',
  'desconocido',
  'desconocido',
  'desconocido',
  'desconocido',
  'desconocido',
  'desconocido',
  'Earth is in a state of constant war and two colonies of humans have been sent to a far away world. The colonists create a new civilization but have inherited the worst traits of their forebears and war breaks out leaving the new citi

ESTÁS TRABAJANDO AQUÍ:

In [15]:

lista_tuplas = []

for i in info_por_peli:
    tupla_datos = [i[0],]
    
    for indice, linea in enumerate(i[1]):
        patron = re.compile(r'([a-z])([A-Z])')

        # directores    
        if 'Dirección' in linea:
            try:                       
                directores = patron.sub(r'\1, \2', i[1][indice+1])
                tupla_datos.append(directores)            

            except:
                tupla_datos.append(i[1][indice+1])
                print('No estoy funcionado')
        # guionistas
        elif 'Guión' in linea:
            try:
                
                guionistas = patron.sub(r'\1, \2', i[1][indice+1])
                tupla_datos.append(guionistas)
                
            except:
                tupla_datos.append(i[1][indice+1])
                print('No estoy funcionado')
        
        # nota y duracion
        elif 'PUNTUACIÓN EN IMDb' in linea:
            tupla_datos.append(float(i[1][indice+1].replace(',', '.')))
            tupla_datos.append(i[1][indice-1])
                    
        # argumento
        elif 'Añade un argumento en tu idioma' in linea:
            tupla_datos.append(i[1][indice+1])
    
    lista_tuplas.append(tuple(tupla_datos))
    

codigo bueno 100% este funciona (afirmación)

In [4]:
with open('../Fase1_datos/lista_drama.json', 'r') as fichero:
    busqueda_drama = json.load(fichero)

In [5]:
busqueda = busqueda_drama[:50]

In [9]:
# llegar a web:
driver = webdriver.Chrome()
driver.get('https://www.imdb.com/')
driver.maximize_window()
sleep(random.randint(1,3))


# rechazar cookies
driver.find_element('css selector', '#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-hLBbgP.sc-ftTHYK.dcvrLS.dufgkr.ecppKW').click()
sleep(random.randint(1,3))


# variables donde iremos almacenando todo
info_por_peli = []


# entrando en cada título
for peli in busqueda:
    lista_info = [f'{peli[0]}',]

    try:
    # metemos el ID en el buscador
        try:
            driver.find_element('css selector', '#suggestion-search').send_keys(peli[0], Keys.ENTER)
            sleep(random.randint(1,3))

            # extraemos info
            info = driver.find_element('xpath' , f'//*[@id="__next"]/main/div/section[1]/section/div[3]/section').text
            lista_info.append(info.split('\n'))

            info_por_peli.append(lista_info)
        except:
            driver.find_element('css selector', '#error > div.error_attrib > a').click()
            
    except:
        fallos.append(peli)


# recogida de los datos
for pelicula in info_por_peli:
    patron = re.compile(r'([a-z])([A-Z])')
    tupla_datos = [pelicula[0]]
    pelicula_Ser = pd.Series(pelicula[1])

    for elemento in ['Dirección', 'Guión', 'PUNTUACIÓN EN IMDb', 'Añade un argumento en tu idioma']:

        if pd.Series(elemento).isin(pelicula_Ser)[0]:
            indice = pelicula_Ser[pelicula_Ser == elemento].index[0]

            if elemento in ['Dirección', 'Guión']:
                nombre = patron.sub(r'\1, \2', pelicula[1][indice+1])
                tupla_datos.append(nombre)
            elif elemento == 'PUNTUACIÓN EN IMDb':
                tupla_datos.append(float(pelicula[1][indice+1].replace(',', '.')))
                tupla_datos.append(pelicula[1][indice-1])
            else:
                tupla_datos.append(pelicula[1][indice+1])

    lista_tuplas.append(tupla_datos)

    lista_tuplas.append(tupla_datos)

NameError: name 'patron' is not defined

In [38]:
fallos = []
lista_tuplas = []

In [42]:
import numpy as np

In [43]:
info_por_peli

[['tt0113092',
  ['Reparto y equipo',
   'Reseñas de usuarios',
   'Curiosidades',
   'IMDbPro',
   'Todos los temas',
   'Por la causa',
   'Título original: For the Cause',
   '2000',
   'R',
   '1h 40min',
   'PUNTUACIÓN EN IMDb',
   '3,4',
   '/10',
   '841',
   'TU PUNTUACIÓN',
   'Puntuar',
   'Acción',
   'Aventura',
   'Drama',
   'Añade un argumento en tu idioma',
   'Earth is in a state of constant war and two colonies of humans have been sent to a far away world. The colonists create a new civilization but have inherited the worst traits of their forebears and war breaks out leaving the new cities in ruins. It falls to a bunch of teens to bring peace to the colony - but will they?',
   'Dirección',
   'David DouglasTim Douglas',
   'Guión',
   'David DouglasChristopher Salazar',
   'Reparto principal',
   'Dean CainThomas Ian GriffithJustin Whalin',
   'Ver la información de la producción en IMDbPro',
   'Añade a mi lista de seguimiento',
   '9',
   'Reseñas de usuarios',
  

In [ ]:
lista_tuplas = []

for peli in info_por_peli:
    
    patron = re.compile(r'([a-z])([A-Z])')

        
    for indice, linea in enumerate(peli[1]):
        tupla_datos = [peli[0],]        
        # listas intermedias para ir checkeando
        direccion = []
        guion = []
        nota = []
        duracion = []
        argumento = []

        # directores    
        if 'Dirección' in linea:
                                
            directores = patron.sub(r'\1, \2', peli[1][indice+1])
            direccion.append(directores)            


        # guionistas
        elif 'Guión' in linea:
            
            guionistas = patron.sub(r'\1, \2', peli[1][indice+1])
            guion.append(guionistas)
                
        
        # nota y duracion
        elif 'PUNTUACIÓN EN IMDb' in linea:
            nota.append(float(peli[1][indice+1].replace(',', '.')))
            duracion.append(peli[1][indice-1])
            
                    
        # argumento
        elif 'Añade un argumento en tu idioma' in linea:
            argumento.append(peli[1][indice+1])

esta es la que funciona

In [96]:
lista_tuplas = []

for peli in info_por_peli:
    
    patron = re.compile(r'([a-z])([A-Z])')

        
    for indice, linea in enumerate(peli[1]):
        tupla_datos = [peli[0],]        
        # listas intermedias para ir checkeando
        direccion = []
        guion = []
        nota = []
        duracion = []
        argumento = []

        # directores    
        if 'Dirección' in linea:
                                
            directores = patron.sub(r'\1, \2', peli[1][indice+1])
            direccion.append(directores)            


        # guionistas
        elif 'Guión' in linea:
            
            guionistas = patron.sub(r'\1, \2', peli[1][indice+1])
            guion.append(guionistas)
                
        
        # nota y duracion
        elif 'PUNTUACIÓN EN IMDb' in linea:
            nota.append(float(peli[1][indice+1].replace(',', '.')))
            duracion.append(peli[1][indice-1])
            
                    
        # argumento
        elif 'Añade un argumento en tu idioma' in linea:
            argumento.append(peli[1][indice+1])
    
   
        # añadimos a la tupla los datos de la lista intermedia
        if direccion != []:
                tupla_datos.append(direccion[0])
        else:
            tupla_datos.append(np.nan)
        

        if guion != []:
                tupla_datos.append(guion[0])
        else:
            tupla_datos.append(np.nan)
        

        if nota != []:
                tupla_datos.append(nota[0])
        else:
            tupla_datos.append(np.nan)
        

        if duracion != []:
                tupla_datos.append(duracion[0])
        else:
            tupla_datos.append(np.nan)
        

        if argumento != []:
                tupla_datos.append(argumento[0])
        else:
            tupla_datos.append(np.nan)
    
        # añadimos a la lista general
        lista_tuplas.append(tuple(tupla_datos))
    

In [ ]:
lista_tuplas = []

for peli in info_por_peli:
    
    patron = re.compile(r'([a-z])([A-Z])')

        
    for indice, linea in enumerate(peli[1]):
        tupla_datos = [peli[0],]        
        # listas intermedias para ir checkeando
        intermedia = []


        # directores    
        if 'Dirección' in linea:
                                
            directores = ['director', patron.sub(r'\1, \2', peli[1][indice+1])]
            intermedia.append(directores)            


        # guionistas
        elif 'Guión' in linea:
            
            guionistas = ['guion', patron.sub(r'\1, \2', peli[1][indice+1])]
            intermedia.append(guionistas)
                
        
        # nota y duracion
        elif 'PUNTUACIÓN EN IMDb' in linea:
            puntuacion = ['nota', peli[1][indice+1].replace(',', '.')]
            intermedia.append(puntuacion)

            minutos = ['minutos', peli[1][indice-1]]
            intermedia.append(minutos)
            
                    
        # argumento
        elif 'Añade un argumento en tu idioma' in linea:
            resumen = ['res', peli[1][indice+1]]
            intermedia.append(resumen)
    
   
        # añadimos a la tupla los datos de la lista intermedia
        
    
        # añadimos a la lista general
        lista_tuplas.append(tuple(tupla_datos))
    

In [ ]:
for peli in info_por_peli:
    print(peli)
 

In [117]:
lista_tuplas = []

for peli in info_por_peli:
    tupla_datos = [peli[0]]
    patron = re.compile(r'([a-z])([A-Z])')
    
    # listas auxiliares
    direccion = []
    guion = []
    nota = []
    duracion = []
    argumento = []

    # bucle para extraer datos       
    for indice, linea in enumerate(peli[1]):

        # directores
        if 'Dirección' in linea:
            directores = patron.sub(r'\1, \2', peli[1][indice+1])
            direccion.append(directores)
        
        # guionistas
        elif 'Guión' in linea:
            guionistas = patron.sub(r'\1, \2', peli[1][indice+1])
            guion.append(guionistas)

        # nota y duracion
        elif 'PUNTUACIÓN EN IMDb' in linea:
            nota.append(float(peli[1][indice+1].replace(',', '.')))
            duracion.append(peli[1][indice-1])
            
                    
        # argumento
        elif 'Añade un argumento en tu idioma' in linea:
            argumento.append(peli[1][indice+1])
    
   
        

    # comprobamos si existe el dato
    if direccion != []:
        tupla_datos.append(direccion[0])
    else:
        tupla_datos.append(np.nan)
    

    if guion != []:
        tupla_datos.append(guion[0])
    else:
        tupla_datos.append(np.nan)
    

    if nota != []:
        tupla_datos.append(nota[0])
    else:
        tupla_datos.append(np.nan)
    

    if duracion != []:
        tupla_datos.append(duracion[0])
    else:
        tupla_datos.append(np.nan)

    if argumento != []:
        tupla_datos.append(argumento[0])
    else:
        tupla_datos.append(np.nan)

        

    lista_tuplas.append(tupla_datos)
            

In [118]:
lista_tuplas

[['tt0113092',
  'David Douglas, Tim Douglas',
  'David Douglas, Christopher Salazar',
  3.4,
  '1h 40min',
  'Earth is in a state of constant war and two colonies of humans have been sent to a far away world. The colonists create a new civilization but have inherited the worst traits of their forebears and war breaks out leaving the new cities in ruins. It falls to a bunch of teens to bring peace to the colony - but will they?'],
 ['tt0113086',
  'Michael New',
  nan,
  6.8,
  '2000',
  "Florentino is a young horseman and singer, a free spirit living in the open world without restrictions, in the Venezuelan plains. He confronts the devil in a duel of improvised verses. As time passes, these unspoilt plains start to change. Through Florentino's voice and actions, the plains cannot be conquered. Its culture defies the devil in the eternal fight between ... Leer todo"],
 ['tt0116748',
  'Rakesh Roshan',
  'Sachin Bhowmick, Ravi Kapoor, Sagar Sarhadi',
  3.7,
  '3h',
  "Wealthy Rajiv Sinh

In [79]:
lista_tuplas = []
# recogida de los datos
for pelicula in info_por_peli:
    patron = re.compile(r'([a-z])([A-Z])')
    
    pelicula_Ser = pd.Series(pelicula[1])


    for elemento in ['Dirección', 'Guión', 'PUNTUACIÓN EN IMDb', 'Añade un argumento en tu idioma']:
        # listas intermedias
        tupla_datos = [pelicula[0]]
        direccion = []
        guion = []
        nota = []
        duracion = []
        argumento = []

        if pd.Series(elemento).isin(pelicula_Ser)[0]:
            indice = pelicula_Ser[pelicula_Ser == elemento].index[0]

            if elemento in ['Dirección']:
                director = patron.sub(r'\1, \2', pelicula[1][indice+1])
                direccion.append(nombre)
           

            elif elemento in ['Guión']:
                guionista = patron.sub(r'\1, \2', pelicula[1][indice+1])
                guion.append(guionista)
            

            elif elemento == 'PUNTUACIÓN EN IMDb':
                nota.append(float(pelicula[1][indice+1].replace(',', '.')))
            
            
            elif elemento == 'PUNTUACIÓN EN IMDb':
                duracion.append(pelicula[1][indice-1])
           

            elif elemento == 'Añade un argumento en tu idioma':
                argumento.append(pelicula[1][indice+1])
            
            
        if direccion != []:
            tupla_datos.append(direccion[0])
        else:
            tupla_datos.append(np.nan)

        
        

    break

        
    